# Bibliotecas

In [1]:
from numpy import *
import sympy as sp
from plotly.graph_objects import *
from plotly.subplots import make_subplots


# 1)

Sejam um círculo definido pelo raio $R$ e centro nas coordenadas do ponto $C$, e uma reta definida de acordo com as coordenadas de um ponto $P$ e um vetor $\vec{v}$, todos no plano $XY$. Utilizando os parâmetros de cada entidade geométrica, avalie através de uma função se a reta é externa, tangente ou secante ao círculo, e caso seja tangente ou secante calcular as coordenadas dos pontos (ou ponto) comuns.

In [2]:
def q1(r, c, p, v):
    theta = sp.Symbol('theta', real=True)
    c_x, c_y, p_x, p_y, v_x, v_y = c[0], c[1], p[0], p[1], v[0], v[1]
    equation = sp.Eq(
        sp.cos(theta)*v_y - sp.sin(theta)*v_x,
        ((c_y - p_y)*v_x - (c_x - p_x)*v_y)/r
    )
    result = sp.solveset(equation, theta, sp.Interval(0, 2*sp.pi, False, True))
    result = array(list(result), dtype=float)
    if result.shape[0] == 0:
        kind = 'externa'
    elif result.shape[0] == 1:
        kind = 'tangente'
    else:
        kind = 'secante'

    t = linspace(0, 2*pi, 100)
    circle = array([c[0]+r*cos(t),c[1]+r*sin(t)])

    interception = array([c[0]+r*cos(result),c[1]+r*sin(result)])

    fig = Figure()
    fig.add_trace(
        Scatter(
            x=[c[0]], y=[c[1]], mode='markers', name='centro', opacity=0.5
        )
    )
    fig.add_trace(
        Scatter(
            x=[p[0]], y=[p[1]], mode='markers', name='ponto', opacity=0.5
        )
    )
    fig.add_trace(
        Scatter(
            x=[p[0],v[0]+p[0]], y=[p[1],v[1]+p[1]], name='vetor', opacity=0.5
        )
    )
    fig.add_trace(
        Scatter(
            x=circle[0], y=circle[1], name='círculo', opacity=0.5
        )
    )
    fig.add_trace(
        Scatter(
            x=interception[0], y=interception[1], name=kind, opacity=0.5
        )
    )
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    fig.show()
    return interception


q1(
    random.rand(), random.rand(2), random.rand(2), random.rand(2)
    #1, [0,0], [0.5,0], [0,1]
)


array([[1.19403226, 0.014442  ],
       [1.40585322, 0.02635322]])

# 2)

Um caminho de uma trajetória possui seu equacionamento a partir da origem do plano cartesiano $(x_0,y_0)=(0,0)$ até o ponto $(x,y)=(4,1)$, dado pela equação:

$$
y=\frac{1}{32}x^\frac{5}{2}
$$

A partir do ponto $(4,1)$ o caminho da trajetória segue uma circunferência até a coordenada $x=7$.

## a)

Qual o raio de curvatura e onde deve estar posicionado o centro desta circunferência, se desejamos manter as continuidades $C_0$ e $C_2$

In [3]:
def q2a():
    theta, a, b, x_c, y_c = sp.symbols('theta a b x_c y_c')
    t = sp.Symbol('t', positive=True)
    # Segmento 1
    r_1 = sp.Matrix(
        [4*t**2,
         t**5,
         0]
    )
    r_1l = sp.diff(r_1)
    r_1ll = sp.diff(r_1l)
    # Curvatura ao final do segmento 1
    r_1l_x_r_1ll = r_1l.cross(r_1ll)
    k_1_num = sp.sqrt(r_1l_x_r_1ll.dot(r_1l_x_r_1ll))
    k_1_den = sp.sqrt(r_1l.dot(r_1l))**3
    k_1 = (k_1_num/k_1_den).subs(t,1)
    # Raio de curvatura no encontro do segmento 1 com o segmento 2
    r = 1/k_1
    # Segmento 2
    r_2 = sp.Matrix(
        [r*sp.cos(a*t+b) + x_c,
         r*sp.sin(a*t+b) + y_c,
         0]
    )
    r_2l = sp.diff(r_2, t)
    # Continuidade C1
    r_2 = r_2.subs(
        b,
        sp.solve(
            sp.Eq(
                r_1l[0].subs(t,1)/r_1l[1].subs(t,1),
                r_2l[0].subs(t,0)/r_2l[1].subs(t,0)
            ),
            b
        )[0]
    )
    # Coordenadas do centro
    x_c_calc = sp.solve(sp.Eq(r_1[0].subs(t,1), r_2[0].subs(t,0)))[0]
    r_2[0] = r_2[0].subs(x_c, x_c_calc)
    y_c_calc = sp.solve(sp.Eq(r_1[1].subs(t,1), r_2[1].subs(t,0)))[0]
    r_2[1] = r_2[1].subs(y_c, y_c_calc)
    # Final do intervalo do segmento 2
    r_2 = r_2.subs(a, sp.solve(sp.Eq(r_2.subs(t,1)[0], 7))[1] - 2*sp.pi)

    fig = Figure()
    fig.add_trace(
        Scatter(
            x=sp.lambdify(t,r_1[0])(linspace(0,1,100)),
            y=sp.lambdify(t,r_1[1])(linspace(0,1,100)),
            name='segmento 1'
        )
    )
    fig.add_trace(
        Scatter(
            x=sp.lambdify(t,r_2[0])(linspace(0,1,100)),
            y=sp.lambdify(t,r_2[1])(linspace(0,1,100)),
            name='segmento 2'
        )
    )
    fig.add_trace(
        Scatter(
            x=[float(r_2.subs(t,0.5)[0]), float(x_c_calc)],
            y=[float(r_2.subs(t,0.5)[1]), float(y_c_calc)],
            name=f'raio = {float(r)}'
        )
    )
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    return fig, r_1, r_1l, r_1ll, r_2, sp.diff(r_2), sp.diff(sp.diff(r_2)), t


q2a()[0].show()


## b)

Avalie a continuidade $C_1$ e as implicações caso sejam diferentes na transição no ponto $(4,1)$

> No item "a" desta questão foi avaliada também a continuidade em $C_1$, pois caso contrário haveria múltiplas soluções. Se fossem avaliadas apenas as continuidades $C_0$ e $C_2$ a curva resultante poderia não ser suave no ponto de encontro dos 2 segmentos.


## c)

Obtenha as expressões para as fórmulas de *Frenet-Serret* para o intervalo $0<x<7$ e verifique se existe alguma descontinuidade.

In [4]:
def q2c():
    s = sp.Symbol('s')
    fig, r_1, r_1l, r_1ll, r_2, r_2l, r_2ll, t = q2a()
    
    # Segmento 1
    k_1 = r_1l.cross(r_1ll).norm()/r_1l.norm()**3
    tal_1 = r_1l.cross(r_1ll).dot(sp.diff(r_1ll))/r_1l.cross(r_1ll).norm()**2
    t_1 = r_1l/r_1l.norm()
    n_1 = sp.diff(t_1)/sp.diff(t_1).norm()
    b_1 = t_1.cross(n_1)
    
    # Segmento 2
    k_2 = r_2l.cross(r_2ll).norm()/r_2l.norm()**3
    tal_2 = r_2l.cross(r_2ll).dot(sp.diff(r_2ll))/r_2l.cross(r_2ll).norm()**2
    t_2 = r_2l/r_2l.norm()
    n_2 = sp.diff(t_2)/sp.diff(t_2).norm()
    b_2 = t_2.cross(n_2)
    
    # Continuidade
    fs_1 = sp.Matrix(
        [[   0,    k_1,     0],
         [-k_1,      0, tal_1],
         [   0, -tal_1,     0]]
    )*sp.Matrix([t_1.T, n_1.T, b_1.T])
    fs_1 = sp.simplify(fs_1.subs(t,1))
    fs_2 = sp.Matrix(
        [[   0,    k_2,     0],
         [-k_2,      0, tal_2],
         [   0, -tal_2,     0]]
    )*sp.Matrix([t_2.T, n_2.T, b_2.T])
    fs_2 = sp.simplify(fs_2.subs(t,0))
    cont = sp.Eq(fs_1, fs_2)
    if cont:
        print('A curva é contínua')
    else:
        print('A curva não é contínua')


q2c()


A curva é contínua


# 3)

Três planos não paralelos entre si, são representados, cada um, por um ponto $P$ e um vetor normal e unitário $\vec{n}$, respectivamente: $P_1$, $\vec{n_1}$; $P_2$, $\vec{n_2}$; $P_3$, $\vec{n_3}$. Tem-se uma esfera de raio $R$ que se apoia nos três planos. Desenvolva o equacionamento para o cálculo do ponto de contato da esfera em cada plano e para o cálculo das coordenadas do centro da esfera. Avalie e discuta o número de soluções para este problema.

In [5]:
def q3():
    a_1, a_2, a_3 = sp.symbols('a_1 a_2 a_3')
    b_1, b_2, b_3 = sp.symbols('b_1 b_2 b_3')
    c_1, c_2, c_3 = sp.symbols('c_1 c_2 c_3')
    x, x_1, x_2, x_3, x_c = sp.symbols('x x_1 x_2 x_3 x_c')
    y, y_1, y_2, y_3, y_c = sp.symbols('y y_1 y_2 y_3 y_c')
    z, z_1, z_2, z_3, z_c = sp.symbols('z z_1 z_2 z_3 z_c')
    r = sp.Symbol('r')
    
    plan1 = sp.Eq(a_1*(x-x_1)+b_1*(y-y_1)+c_1*(z-z_1),0)
    plan2 = sp.Eq(a_2*(x-x_2)+b_2*(y-y_2)+c_2*(z-z_2),0)
    plan3 = sp.Eq(a_3*(x-x_3)+b_3*(y-y_3)+c_3*(z-z_3),0)
    
    center1, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c-a_1*r).subs(y,y_c-b_1*r).subs(z,z_c-c_1*r),
                plan2.subs(x,x_c-a_2*r).subs(y,y_c-b_2*r).subs(z,z_c-c_2*r),
                plan3.subs(x,x_c-a_3*r).subs(y,y_c-b_3*r).subs(z,z_c-c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p11 = sp.Matrix([center1[0]-a_1*r,center1[1]-b_1*r,center1[2]-c_1*r])
    p12 = sp.Matrix([center1[0]-a_2*r,center1[1]-b_2*r,center1[2]-c_2*r])
    p13 = sp.Matrix([center1[0]-a_3*r,center1[1]-b_3*r,center1[2]-c_3*r])

    center2, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c+a_1*r).subs(y,y_c+b_1*r).subs(z,z_c+c_1*r),
                plan2.subs(x,x_c-a_2*r).subs(y,y_c-b_2*r).subs(z,z_c-c_2*r),
                plan3.subs(x,x_c-a_3*r).subs(y,y_c-b_3*r).subs(z,z_c-c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p21 = sp.Matrix([center2[0]+a_1*r,center2[1]+b_1*r,center2[2]+c_1*r])
    p22 = sp.Matrix([center2[0]-a_2*r,center2[1]-b_2*r,center2[2]-c_2*r])
    p23 = sp.Matrix([center2[0]-a_3*r,center2[1]-b_3*r,center2[2]-c_3*r])

    center3, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c-a_1*r).subs(y,y_c-b_1*r).subs(z,z_c-c_1*r),
                plan2.subs(x,x_c+a_2*r).subs(y,y_c+b_2*r).subs(z,z_c+c_2*r),
                plan3.subs(x,x_c-a_3*r).subs(y,y_c-b_3*r).subs(z,z_c-c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p31 = sp.Matrix([center3[0]-a_1*r,center3[1]-b_1*r,center3[2]-c_1*r])
    p32 = sp.Matrix([center3[0]+a_2*r,center3[1]+b_2*r,center3[2]+c_2*r])
    p33 = sp.Matrix([center3[0]-a_3*r,center3[1]-b_3*r,center3[2]-c_3*r])

    center4, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c-a_1*r).subs(y,y_c-b_1*r).subs(z,z_c-c_1*r),
                plan2.subs(x,x_c-a_2*r).subs(y,y_c-b_2*r).subs(z,z_c-c_2*r),
                plan3.subs(x,x_c+a_3*r).subs(y,y_c+b_3*r).subs(z,z_c+c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p41 = sp.Matrix([center4[0]-a_1*r,center4[1]-b_1*r,center4[2]-c_1*r])
    p42 = sp.Matrix([center4[0]-a_2*r,center4[1]-b_2*r,center4[2]-c_2*r])
    p43 = sp.Matrix([center4[0]+a_3*r,center4[1]+b_3*r,center4[2]+c_3*r])

    center5, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c+a_1*r).subs(y,y_c+b_1*r).subs(z,z_c+c_1*r),
                plan2.subs(x,x_c+a_2*r).subs(y,y_c+b_2*r).subs(z,z_c+c_2*r),
                plan3.subs(x,x_c-a_3*r).subs(y,y_c-b_3*r).subs(z,z_c-c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p51 = sp.Matrix([center5[0]+a_1*r,center5[1]+b_1*r,center5[2]+c_1*r])
    p52 = sp.Matrix([center5[0]+a_2*r,center5[1]+b_2*r,center5[2]+c_2*r])
    p53 = sp.Matrix([center5[0]-a_3*r,center5[1]-b_3*r,center5[2]-c_3*r])

    center6, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c+a_1*r).subs(y,y_c+b_1*r).subs(z,z_c+c_1*r),
                plan2.subs(x,x_c-a_2*r).subs(y,y_c-b_2*r).subs(z,z_c-c_2*r),
                plan3.subs(x,x_c+a_3*r).subs(y,y_c+b_3*r).subs(z,z_c+c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p61 = sp.Matrix([center6[0]+a_1*r,center6[1]+b_1*r,center6[2]+c_1*r])
    p62 = sp.Matrix([center6[0]-a_2*r,center6[1]-b_2*r,center6[2]-c_2*r])
    p63 = sp.Matrix([center6[0]+a_3*r,center6[1]+b_3*r,center6[2]+c_3*r])

    center7, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c-a_1*r).subs(y,y_c-b_1*r).subs(z,z_c-c_1*r),
                plan2.subs(x,x_c+a_2*r).subs(y,y_c+b_2*r).subs(z,z_c+c_2*r),
                plan3.subs(x,x_c+a_3*r).subs(y,y_c+b_3*r).subs(z,z_c+c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p71 = sp.Matrix([center7[0]-a_1*r,center7[1]-b_1*r,center7[2]-c_1*r])
    p72 = sp.Matrix([center7[0]+a_2*r,center7[1]+b_2*r,center7[2]+c_2*r])
    p73 = sp.Matrix([center7[0]+a_3*r,center7[1]+b_3*r,center7[2]+c_3*r])

    center8, = sp.linsolve(
        sp.linear_eq_to_matrix(
            (
                plan1.subs(x,x_c+a_1*r).subs(y,y_c+b_1*r).subs(z,z_c+c_1*r),
                plan2.subs(x,x_c+a_2*r).subs(y,y_c+b_2*r).subs(z,z_c+c_2*r),
                plan3.subs(x,x_c+a_3*r).subs(y,y_c+b_3*r).subs(z,z_c+c_3*r)
            ),
            x_c,
            y_c,
            z_c
        ),
        [x_c, y_c, z_c]
    )
    p81 = sp.Matrix([center8[0]+a_1*r,center8[1]+b_1*r,center8[2]+c_1*r])
    p82 = sp.Matrix([center8[0]+a_2*r,center8[1]+b_2*r,center8[2]+c_2*r])
    p83 = sp.Matrix([center8[0]+a_3*r,center8[1]+b_3*r,center8[2]+c_3*r])

    v1 = random.uniform(-1,1,3)
    v1 = v1/sqrt(v1.dot(v1))
    while True:
        v2 = random.uniform(-1,1,3)
        v2 = v2/sqrt(v2.dot(v2))
        if array_equal(v1, v2) or array_equal(v1, -v2):
            None
        else:
            break
    while True:
        v3 = random.uniform(-1,1,3)
        v3 = v3/sqrt(v3.dot(v3))
        if array_equal(v1, v3) or array_equal(v1, -v3) or array_equal(v2, v3) or array_equal(v2, -v3):
            None
        else:
            break
    v = array([v1,v2,v3])
    p = random.uniform(-1,1,(3,3))
    radius = random.uniform(0.1,1)
    contact_points = sp.Matrix(
        [p11.T, p12.T, p13.T, p21.T, p22.T, p23.T, p31.T, p32.T, p33.T, p41.T, p42.T, p43.T,
         p51.T, p52.T, p53.T, p61.T, p62.T, p63.T, p71.T, p72.T, p73.T, p81.T, p82.T, p83.T]
    )
    contact_points = sp.lambdify(
        (a_1, b_1, c_1, a_2, b_2, c_2, a_3, b_3, c_3, x_1, y_1, z_1, x_2, y_2, z_2, x_3, y_3, z_3, r),
        contact_points
    )(v[0,0], v[0,1], v[0,2], v[1,0], v[1,1], v[1,2], v[2,0], v[2,1], v[2,2], p[0,0], p[0,1], p[0,2], p[1,0], p[1,1], p[1,2], p[2,0], p[2,1], p[2,2], radius)

    center_sphere = sp.Matrix(
        [sp.Matrix(center1).T, sp.Matrix(center2).T, sp.Matrix(center3).T, sp.Matrix(center4).T,
         sp.Matrix(center5).T, sp.Matrix(center6).T, sp.Matrix(center7).T, sp.Matrix(center8).T, ]
    )
    center_sphere = sp.lambdify(
        (a_1, b_1, c_1, a_2, b_2, c_2, a_3, b_3, c_3, x_1, y_1, z_1, x_2, y_2, z_2, x_3, y_3, z_3, r),
        center_sphere
    )(v[0,0], v[0,1], v[0,2], v[1,0], v[1,1], v[1,2], v[2,0], v[2,1], v[2,2], p[0,0], p[0,1], p[0,2], p[1,0], p[1,1], p[1,2], p[2,0], p[2,1], p[2,2], radius)

    u, w = meshgrid(linspace(0, 1, 50), linspace(0, 1, 50))

    fig = Figure()
    fig.add_trace(
        Cone(
            x=p[:,0],
            y=p[:,1],
            z=p[:,2],
            u=v[:,0],
            v=v[:,1],
            w=v[:,2],
            text=['n1', 'n2', 'n3'],
            sizeref=1,
            sizemode="scaled",
            anchor="tail",
            opacity=0.5,
            name='vetores',
            showscale=False
        )
    )
    fig.add_trace(
        Scatter3d(
            x=p[:,0],
            y=p[:,1],
            z=p[:,2],
            mode='markers',
            name='pontos'
        )
    )
    fig.add_trace(
        Scatter3d(
            x=contact_points[:,0],
            y=contact_points[:,1],
            z=contact_points[:,2],
            mode='markers',
            name='pontos de contato'
        )
    )
    fig.add_trace(
        Mesh3d(
            x=append(contact_points[::3,0], p[0,0]),
            y=append(contact_points[::3,1], p[0,1]),
            z=append(contact_points[::3,2], p[0,2]),
            name='planos'
        )
    )
    fig.add_trace(
        Mesh3d(
            x=append(contact_points[1::3,0], p[1,0]),
            y=append(contact_points[1::3,1], p[1,1]),
            z=append(contact_points[1::3,2], p[1,2]),
            name='planos'
        )
    )
    fig.add_trace(
        Mesh3d(
            x=append(contact_points[2::3,0], p[2,0]),
            y=append(contact_points[2::3,1], p[2,1]),
            z=append(contact_points[2::3,2], p[2,2]),
            name='planos'
        )
    )
    for n in range(8):
        sphere = array(
            [radius*sin(pi*u)*cos(2*pi*w) + center_sphere[n,0],
            radius*sin(pi*u)*sin(2*pi*w) + center_sphere[n,1],
            radius*cos(pi*u) + center_sphere[n,2]]
        )
        fig.add_trace(
            Surface(
                x=sphere[0],
                y=sphere[1],
                z=sphere[2],
                name='esferas',
                showscale=False
            )
        )
    fig.show()


q3()


# 4)

Uma curva de Bezier cúbica, com continuidade $C_0$, $C_1$ e $C_2$, possui os seguintes pontos de controle para cada segmento:

|  | Pontos de Controle | Coordenadas $$({V_{j}^{i}}_{x},{V_{j}^{i}}_{y},{V_{j}^{i}}_{x})$$ |
|:---:|:---:|:---:|
| Segmento 0 | $$V_{0}^{0}$$ $$V_{1}^{0}$$ $$V_{2}^{0}$$ $$V_{3}^{0}$$ | $$(1.00000, 2.00000, 1.00000)$$ $$(1.36083, 1.63917, 0.87766)$$ $$(1.69535, 1.35223, 0.83137)$$ $$(2.00000, 1.00000, 1.00000)$$ |
| Segmento 1 | $$V_{0}^{1}$$ $$V_{1}^{1}$$ $$V_{2}^{1}$$ $$V_{3}^{1}$$ | $$(2.00000, 1.00000, 1.00000)$$ $$(2.37312, 0.56861, 1.20653)$$ $$(2.70143, 0.03931, 1.73544)$$ $$(3.00000, 0.00000, 2.00000)$$ |
| Segmento 2 | $$V_{0}^{2}$$ $$V_{1}^{2}$$ $$V_{2}^{2}$$ $$V_{3}^{2}$$ | $$(3.00000, 0.00000, 2.00000)$$ $$(3.38545, -0.05075, 2.34154)$$ $$(3.72133, 0.71517, 2.24250)$$ $$(4.00000, 2.00000, 2.00000)$$ |

## a)

Desenvolva uma função que calcule novos pontos de controle que correspondam a divisão de cada segmento em $u=u_d$ via De Casteljau.

In [6]:
def q4a():
    seg0 = sp.Matrix(
        [[1.00000,2.00000,1.00000],
         [1.36083,1.63917,0.87766],
         [1.69535,1.35223,0.83137],
         [2.00000,1.00000,1.00000]]
    )
    seg1 = sp.Matrix(
        [[2.00000,1.00000,1.00000],
         [2.37312,0.56861,1.20653],
         [2.70143,0.03931,1.73544],
         [3.00000,0.00000,2.00000]]
    )
    seg2 = sp.Matrix(
        [[3.00000,0.00000,2.00000],
         [3.38545,-0.05075,2.34154],
         [3.72133,0.71517,2.24250],
         [4.00000,2.00000,2.00000]]
    )
    

    def de_casteljau_split(control_points):
        u_d = sp.Symbol('u_d')
        v01 = (1-u_d)*control_points[0,:] + u_d*control_points[1,:]
        v12 = (1-u_d)*control_points[1,:] + u_d*control_points[2,:]
        v23 = (1-u_d)*control_points[2,:] + u_d*control_points[3,:]

        v02 = (1-u_d)*v01 + u_d*v12
        v13 = (1-u_d)*v12 + u_d*v23

        v03 = (1-u_d)*v02 + u_d*v13

        seg0 = sp.Matrix(
            [control_points[0,:],
             v01,
             v02,
             v03]
        )
        seg1 = sp.Matrix(
            [v03,
             v13,
             v23,
             control_points[3,:]]
        )
        
        return seg0, seg1
    

    new_seg0, new_seg1 = de_casteljau_split(seg0)
    new_seg2, new_seg3 = de_casteljau_split(seg1)
    new_seg4, new_seg5 = de_casteljau_split(seg2)

    new_control_points = sp.Matrix(
        [new_seg0,
         new_seg1,
         new_seg2,
         new_seg3,
         new_seg4,
         new_seg5,]
    )
    return new_control_points


sp.simplify(q4a())


Matrix([
[                                                                        1.0,                                                  2.0,                                                  1.0],
[                                                          0.36083*u_d + 1.0,                                    2.0 - 0.36083*u_d,                                    1.0 - 0.12234*u_d],
[                             -0.0263100000000001*u_d**2 + 0.72166*u_d + 1.0,                   0.07389*u_d**2 - 0.72166*u_d + 2.0,        0.0760500000000001*u_d**2 - 0.24468*u_d + 1.0],
[-0.00355999999999979*u_d**3 - 0.0789300000000002*u_d**2 + 1.08249*u_d + 1.0, -0.13918*u_d**3 + 0.22167*u_d**2 - 1.08249*u_d + 2.0,  0.13887*u_d**3 + 0.22815*u_d**2 - 0.36702*u_d + 1.0],
[-0.00355999999999979*u_d**3 - 0.0789300000000002*u_d**2 + 1.08249*u_d + 1.0, -0.13918*u_d**3 + 0.22167*u_d**2 - 1.08249*u_d + 2.0,  0.13887*u_d**3 + 0.22815*u_d**2 - 0.36702*u_d + 1.0],
[                         -0.0298699999999998*u_d**2 + 0

## b)

Considere um ponto $P$ com coordenadas $(4.000, 0.000, 2.000)$. Utilizando o método desenvolvido acima, e fazendo $u_d=0.5$, encontre sobre a curva o ponto mais próximo de $P$.

In [7]:
def q4b(res=50, point=array([4,0,2])):
    point = array(point)
    u = linspace(0,1,res)
    us = array(
        [1+u*0, u, u**2, u**3]
    )
    mbz = array(
        [[ 1,  0,  0,  0],
         [-3,  3,  0,  0],
         [ 3, -6,  3,  0],
         [-1,  3, -3,  1]]
    )
    u_d = sp.Symbol('u_d')
    control_points = array(q4a().subs(u_d, 0.5), dtype=float)
    n_seg = int(control_points.shape[0]/4)
    curve = zeros((res*n_seg,4))
    for n in range(n_seg):
        curve[n*res:(n+1)*res,:3] = matmul(
            matmul(us.T, mbz),
            control_points[n*4:(n+1)*4]
        )
    for n in range(curve.shape[0]):
        curve[n,3] = linalg.norm(curve[n,:3]-point)

    min_dist = curve[curve[:,3] == curve[:,3].min()]

    fig = Figure()
    fig.add_trace(
        Scatter3d(
            x=curve[:,0],
            y=curve[:,1],
            z=curve[:,2],
            mode='lines',
            name='curva'
        )
    )
    fig.add_trace(
        Scatter3d(
            x=control_points[:,0],
            y=control_points[:,1],
            z=control_points[:,2],
            name='pontos de controle'
        )
    )
    for n in range(min_dist.shape[0]):
        fig.add_trace(
            Scatter3d(
                x=[point[0], min_dist[n,0]],
                y=[point[1], min_dist[n,1]],
                z=[point[2], min_dist[n,2]],
                name=f'distância {min_dist[n,3]}'
            )
        )
    fig.show()


q4b()


# 5)

Considere a formulação de uma curva de Bézier cúbica:

## a)

Obtenha o equacionamento de um segmento de curva a partir da definição das derivadas de segunda ordem ($M_0$ e $M_1$ - derivadas de 2ª ordem) nas origens do segmento (diferentemente do equacionamento apresentado em aula, onde foi equacionada a Bézier a partir das derivadas de primeira ordem $m_0$ e $m_1$)

In [8]:
def q5a():
    M = sp.IndexedBase('M')
    r = sp.IndexedBase('r')
    V = sp.IndexedBase('V')
    eq1 = sp.Eq(M[0], 6*(r[0] - 2*V[1] + V[2]))
    eq2 = sp.Eq(M[1], 6*(V[1] - 2*V[2] + r[1]))
    solution = sp.solve([eq1, eq2], [V[1], V[2]])
    eq_f = sp.Eq(
        sp.Matrix([V[0], V[1], V[2], V[3]]),
        sp.Matrix([r[0], solution[V[1]], solution[V[2]], r[1]])
    )
    return eq_f


q5a()


Eq(Matrix([
[V[0]],
[V[1]],
[V[2]],
[V[3]]]), Matrix([
[                                 r[0]],
[-M[0]/9 - M[1]/18 + 2*r[0]/3 + r[1]/3],
[-M[0]/18 - M[1]/9 + r[0]/3 + 2*r[1]/3],
[                                 r[1]]]))

## b)

Dada uma sequência de pontos em coordenadas espaciais ($r_0$, $r_1$, $...$, $r_n$), desenvolva o equacionamento da curva de Bézier composta utilizando o equacionamento do segmento a partir das derivadas de segunda ordem.

> $$
\begin{bmatrix}
18 & 0 & 0 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & k_{1,1} & 1 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0\\ 
-k_{1,2} & 2k_{1,2} & -2 & 1 & 0 & 0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & k_{2,1} & 1 & 0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & -k_{2,2} & 2k_{2,2} & -2 & 1 & \cdots & 0 & 0 & 0 & 0 & 0 & 0\\ 
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\ 
0 & 0 & 0 & 0 & 0 & 0 & \cdots & 0 & k_{n-2,1} & 1 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & \cdots & -k_{n-2,2} & 2k_{n-2,2} & -2 & 1 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & k_{n-1,1} & 1 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & \cdots & 0 & 0 & -k_{n-1,2} & 2k_{n-1,2} & -2 & 1\\ 
0 & 0 & 0 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & 0 & 0 & 18
\end{bmatrix}
\begin{bmatrix}
V_{1,1}\\ 
V_{1,2}\\ 
V_{2,1}\\ 
V_{2,2}\\ 
V_{3,1}\\ 
V_{3,2}\\ 
\vdots\\ 
V_{n-2,1}\\ 
V_{n-2,2}\\ 
V_{n-1,1}\\ 
V_{n-1,2}\\ 
V_{n,1}\\ 
V_{n,2}
\end{bmatrix}
=
\begin{bmatrix}
-2M_{0} -M_{1} +12r_{0} +6r_{1}\\ 
r_{1}(k_{1,1}+1)\\ 
r_{1}(k_{1,2}-1)\\ 
r_{2}(k_{2,1}+1)\\ 
r_{2}(k_{2,2}-1)\\ 
\vdots\\ 
r_{n-2}(k_{n-2,1}+1)\\ 
r_{n-2}(k_{n-2,2}-1)\\ 
r_{n-1}(k_{n-1,1}+1)\\ 
r_{n-1}(k_{n-1,2}-1)\\ 
-M_{n-1} -2M_{n} +6r_{n-1} +12r_{n}
\end{bmatrix}
$$

# 6)

Desenvolva a formulação Bezier e B-Spline cúbica para uma curva fechada (onde $r_0=r_n$), a qual devem ser mantidas as continuidades no fechamento. Avalie o comportamento do erro de aproximação na tentativa de ajustarmos uma circunferência de raio $R$ através de uma curva de Bézier, para 4, 8 e 12 pontos conhecidos da circunferência.

In [9]:
def q6bz(*args):
    res = 50
    t = linspace(0, 1, res)
    circle = array([cos(2*pi*t), sin(2*pi*t)])
    error = {}
    R, u = sp.symbols('R u')
    fig = make_subplots(
        rows=1,
        cols=2,
        shared_xaxes='all',
        shared_yaxes='all',
        subplot_titles=(['pontos (1ª derivada)', 'pontos (2ª derivada)']),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}]]
    )
    for deriv in range(1,3):
        for arg in enumerate(args):
            a = sp.zeros(arg[1]*2)
            b = sp.zeros(arg[1]*2, 2)
            if deriv == 1:
                a[0,0], a[-1,-1] = 3, 3
                b[0,:] = sp.Matrix([3*R, 2*sp.pi*R/arg[1]]).T
                b[-1,:] = sp.Matrix([3*R, -2*sp.pi*R/arg[1]]).T
            elif deriv == 2:
                a[0,0], a[-1,-1] = 18, 18
                b[0,:] = sp.Matrix(
                    [2*R*(2*sp.pi**2/arg[1]**2 +3)*(sp.cos(2*sp.pi/arg[1])+2),
                    2*R*(2*sp.pi**2/arg[1]**2 +3)*(sp.sin(2*sp.pi/arg[1]))]
                ).T
                b[-1,:] = sp.Matrix(
                    [2*R*(2*sp.pi**2/arg[1]**2 +3)*(sp.cos(-2*sp.pi/arg[1])+2),
                    2*R*(2*sp.pi**2/arg[1]**2 +3)*(sp.sin(-2*sp.pi/arg[1]))]
                ).T
            else:
                None
            curve_eqs = sp.zeros(arg[1], 2)
            curve = zeros((arg[1]*res,2))
            for n in range(arg[1]-1):
                a[2*n+1,2*n+1], a[2*n+1,2*n+2] = 1, 1
                a[2*n+2,2*n+0], a[2*n+2,2*n+1] = -1, 2
                a[2*n+2,2*n+2], a[2*n+2,2*n+3] = -2, 1
                b[2*n+1,0] = 2*R*sp.cos(2*sp.pi*(n+1)/arg[1])
                b[2*n+1,1] = 2*R*sp.sin(2*sp.pi*(n+1)/arg[1])
            v_x, = sp.linsolve((a,b[:,0]))
            v_y, = sp.linsolve((a,b[:,1]))
            v = sp.Matrix([v_x, v_y]).T
            for n in range(arg[1]+1):
                v = v.row_insert(
                    3*n,
                    sp.Matrix(
                        [R*sp.cos(2*sp.pi*n/arg[1]), R*sp.sin(2*sp.pi*n/arg[1])]
                    ).T
                )
            for n in range(arg[1]):
                curve_eqs[n,:] = sp.Matrix(
                    [1,u,u**2,u**3]
                ).T*sp.Matrix(
                    [[ 1,  0,  0,  0],
                    [-3,  3,  0,  0],
                    [ 3, -6,  3,  0],
                    [-1,  3, -3,  1]]
                )*v[3*n:3*(n+1)+1,:]
                curve[n*res:(n+1)*res,0] = sp.lambdify(
                    u, curve_eqs[n,0].subs(R,1)
                )(t)
                curve[n*res:(n+1)*res,1] = sp.lambdify(
                    u, curve_eqs[n,1].subs(R,1)
                )(t)
            
            e = 0
            for n in range(curve.shape[0]):
                e += (sqrt(curve[n,0]**2 + curve[n,1]**2) - 1)**2
            e /= curve.shape[0]
            
            error[f'{arg[1]} C{deriv}'] = sqrt(e)

            fig.add_trace(
                Scatter(
                    x=curve[:,0],
                    y=curve[:,1],
                    name=f'bezier ({arg[1]} C{deriv})',
                    opacity=0.5
                ),
                row=1,
                col=deriv
            )
        fig.add_trace(
            Scatter(
                x=circle[0],
                y=circle[1],
                name=f'parametrizado (C{deriv})',
                opacity=0.5
            ),
            row=1,
            col=deriv
        )
    fig.update_yaxes(scaleanchor = 'x')
    fig.show()
    return error


q6bz(4, 8, 12)


{'4 C1': 0.015973229285326285,
 '8 C1': 0.0007163653301200217,
 '12 C1': 0.0001314892334709401,
 '4 C2': 0.029893129405255996,
 '8 C2': 0.0013466293871845134,
 '12 C2': 0.00022797949058668844}

In [10]:
def q6bs(*args):
    res = 50
    t = linspace(0, 1, res)
    circle = array([cos(2*pi*t), sin(2*pi*t)])
    error = {}
    R, u = sp.symbols('R u')
    fig = Figure()
    col=1
    for arg in args:
        a = sp.zeros(arg)
        a = a + sp.eye(arg)
        a[:-1,1:] = a[:-1,1:] + sp.eye(arg-1)*4
        a[-1:,:1] = a[-1:,:1] + sp.eye(1)*4
        a[:-2,2:] = a[:-2,2:] + sp.eye(arg-2)
        a[-2:,:2] = a[-2:,:2] + sp.eye(2)
        a = a/6
        b = sp.zeros(arg, 2)
        for n in range(arg):
            b[n,:] = sp.Matrix(
                [R*sp.cos(2*sp.pi*n/arg), R*sp.sin(2*sp.pi*n/arg)]
            ).T
        v_x, = sp.linsolve((a,b[:,0]))
        v_y, = sp.linsolve((a,b[:,1]))
        v = sp.Matrix([v_x, v_y]).T
        v = v.row_insert(arg+0, v[0,:])
        v = v.row_insert(arg+1, v[1,:])
        v = v.row_insert(arg+2, v[2,:])
        curve_eqs = sp.zeros(arg,2)
        curve = zeros((arg*res,2))
        for n in range(arg):
            curve_eqs[n,:] = sp.Matrix(
                [1,u,u**2,u**3]
            ).T*sp.Matrix(
                [[ 1,  4,  1,  0],
                 [-3,  0,  3,  0],
                 [ 3, -6,  3,  0],
                 [-1,  3, -3,  1]]
            )*v[n:n+4,:]/6
            curve[n*res:(n+1)*res,0] = sp.lambdify(
                u, curve_eqs[n,0].subs(R,1)
            )(t)
            curve[n*res:(n+1)*res,1] = sp.lambdify(
                u, curve_eqs[n,1].subs(R,1)
            )(t)
        
        e = 0
        for n in range(curve.shape[0]):
            e += (sqrt(curve[n,0]**2 + curve[n,1]**2) - 1)**2
        e /= curve.shape[0]
        
        error[arg] = sqrt(e)

        fig.add_trace(
            Scatter(
                x=curve[:,0],
                y=curve[:,1],
                name=f'bspline ({arg})',
                opacity=0.5
            )
        )
        col += 1
    fig.add_trace(
        Scatter(
            x=circle[0],
            y=circle[1],
            name=f'parametrizado',
            opacity=0.5
        )
    )
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    fig.show()
    return error


q6bs(4, 8, 12)


{4: 0.01723324823237171, 8: 0.0007236278269231887, 12: 0.00013191247815471033}

# 7)

Sejam dois segmentos cúbicos de curva de Bezier distintos e quaisquer identificados como: $Seg_1$ e $Seg_2$; e que são definidos pelos seus respectivos pontos de controle. Calcule uma curva Bezier intermediária (especificamente os pontos de controle) que conecta o final do segmento 1 ($Seg_1$) com o início do segmento 2 ($Seg_2$) mantendo as continuidades de $C_0$, $C_1$ e $C_2$ nas transições entre segmentos.

In [11]:
def q7(seg1, seg3, k1, k2):
    seg1 = array(seg1)
    seg2 = zeros((4,3))
    seg3 = array(seg3)
    
    seg2[0] = seg1[3]
    seg2[3] = seg3[0]
    seg2[1] = seg2[0] + k1*(seg1[3] - seg1[2])
    seg2[2] = seg2[3] - k1*(seg3[1] - seg3[0])

    u = linspace(0, 1, 1000)

    r1 = array(0)
    for i in range(3+1):
        c = 6/(math.factorial(i)*math.factorial(3-i))
        r1 = r1 + multiply.outer(c*(u**i)*((1-u)**(3-i)), seg1[i])
    
    r2 = array(0)
    for i in range(3+1):
        c = 6/(math.factorial(i)*math.factorial(3-i))
        r2 = r2 + multiply.outer(c*(u**i)*((1-u)**(3-i)), seg2[i])
    
    r3 = array(0)
    for i in range(3+1):
        c = 6/(math.factorial(i)*math.factorial(3-i))
        r3 = r3 + multiply.outer(c*(u**i)*((1-u)**(3-i)), seg3[i])
    
    fig = Figure()
    fig.add_trace(
        Scatter3d(
            x=r1[:,0],
            y=r1[:,1],
            z=r1[:,2],
            mode='lines',
            name='segmento 1'
        )
    )
    fig.add_trace(
        Scatter3d(
            x=seg1[:,0],
            y=seg1[:,1],
            z=seg1[:,2],
            opacity=0.5,
            name='pontos 1'
        )
    )
    fig.add_trace(
        Scatter3d(
            x=r2[:,0],
            y=r2[:,1],
            z=r2[:,2],
            mode='lines',
            name='segmento 2'
        )
    )
    fig.add_trace(
        Scatter3d(
            x=seg2[:,0],
            y=seg2[:,1],
            z=seg2[:,2],
            opacity=0.5,
            name='pontos 2'
        )
    )
    fig.add_trace(
        Scatter3d(
            x=r3[:,0],
            y=r3[:,1],
            z=r3[:,2],
            mode='lines',
            name='segmento 3'
        )
    )
    fig.add_trace(
        Scatter3d(
            x=seg3[:,0],
            y=seg3[:,1],
            z=seg3[:,2],
            opacity=0.5,
            name='pontos 3'
        )
    )
    fig.show()


q7(
    random.rand(4,3),
    random.rand(4,3),
    random.rand()+0.5,
    random.rand()+0.5,
)


# 8)

Uma esfera de raio $R=4$ é parametrizada por $r(u,w)$ como segue:

$$
r(u,w)=
\begin{matrix}
\left\{\begin{matrix}
4\sin(u)\cos(w)\\ 
4\sin(u)\sin(w)\\ 
4\cos(u)
\end{matrix}\right.
&
\text{para: } 0<u<\pi \text{ e } 0<w<2\pi
\end{matrix}
$$

## a)

Avalie a curvatura de um caminho sobre a esfera, através do equacionamento obtido pela relação entre a 1ª e 2ª formas fundamentais, para qualquer caminho sobre a superfície. Observe que o resultado final deverá ser $k=\frac{1}{4}$, que corresponde ao inverso do raio de curvatura da esfera.

In [12]:
def q8a():
    t = sp.Symbol('t')
    u = sp.Function('u')(t)
    w = sp.Function('w')(t)
    r = 4*sp.Matrix(
        [sp.sin(u)*sp.cos(w),
         sp.sin(u)*sp.sin(w),
         sp.cos(u)]
    )
    ru = sp.diff(r, u)
    ruu = sp.diff(ru, u)
    rw = sp.diff(r, w)
    rww = sp.diff(rw, w)
    ruw = sp.diff(ru, w)
    g = sp.Matrix(
        [[ru.dot(ru), ru.dot(rw)],
         [ru.dot(rw), rw.dot(rw)]]
    )
    ru_x_rw = ru.cross(rw)
    n = ru_x_rw/sp.sqrt(ru_x_rw.dot(ru_x_rw))
    d = sp.Matrix(
        [[n.dot(ruu), n.dot(ruw)],
         [n.dot(ruw), n.dot(rww)]]
    )
    vec = sp.Matrix([sp.diff(u, t),sp.diff(w, t)])
    num = vec.T*d*vec
    den = vec.T*g*vec
    return sp.simplify(num[0]/den[0]), t, u, w, r, ru, rw


q8a()[0]


-sin(u(t))/(4*sqrt(sin(u(t))**2))

## b)

Sobre esta esfera, define-se um caminho inicial para $u$ e $w$ parametrizado em $t$ cuja parametrização é dada por:

$$
\begin{matrix}
\left\{\begin{matrix}
u(t)=\pi t^2\\ 
w(t)=2\pi(t^2-1)
\end{matrix}\right.
&
\text{para: } 0<t<1
\end{matrix}
$$

Desenvolva um programa que apresente graficamente a esfera e o caminho inicial definido.

In [13]:
def q8b(res=50):
    t = linspace(0,1,res)
    u, w = meshgrid(pi*t**2, 2*pi*(t**2-1))
    sphere = 4*array(
        [sin(u)*cos(w),
         sin(u)*sin(w),
         cos(u)]
    )
    t = linspace(0,1,res*100)
    u = pi*t**2
    w = 2*pi*(t**2-1)
    path = 4*array(
        [sin(u)*cos(w),
         sin(u)*sin(w),
         cos(u)]
    )
    fig = Figure()
    fig.add_trace(
        Surface(
            x=sphere[0],
            y=sphere[1],
            z=sphere[2],
            showscale=False
        )
    )
    fig.add_trace(
        Scatter3d(
            x=path[0],
            y=path[1],
            z=path[2],
            mode='lines',
            line=scatter3d.Line(width=5)
        )
    )
    #fig.show()
    return fig


q8b().show()


## c)

Calcule o caminho paralelo e apresente-o juntamente com o caminho inicial sobre a superfície esférica.

In [14]:
def q8c(res=50):
    dummy, t, u, w, r, ru, rw = q8a()
    g = .5
    E = ru.dot(ru).subs(u, sp.pi*t**2).subs(w, 2*sp.pi*(t**2-1)).simplify()
    F = ru.dot(rw).subs(u, sp.pi*t**2).subs(w, 2*sp.pi*(t**2-1)).simplify()
    G = rw.dot(rw).subs(u, sp.pi*t**2).subs(w, 2*sp.pi*(t**2-1)).simplify()
    ul = sp.diff(sp.pi*t**2)
    wl = sp.diff(2*sp.pi*(t**2-1))
    den = sp.sqrt(E*G-F**2)*sp.sqrt(E*ul**2+2*F*ul*wl+G*wl**2).simplify()
    delta_u = g*(E*ul+F*wl)/den
    delta_w = -g*(F*ul+G*wl)/den
    r = r.subs(
        u, sp.pi*t**2+delta_u
    ).subs(
        w, 2*sp.pi*(t**2-1)+delta_w
    )
    fig = q8b()
    fig.add_trace(
        Scatter3d(
            x=sp.lambdify(t,r[0])(linspace(.3,.9,100)),
            y=sp.lambdify(t,r[1])(linspace(.3,.9,100)),
            z=sp.lambdify(t,r[2])(linspace(.3,.9,100)),
            mode='lines',
            name='paralelo',
            line=scatter3d.Line(width=5)
        )
    )
    fig.show()


q8c()


# 9)

Discuta as seguintes questões:

## a)

Quais as consequências no equacionamento de uma superfície, a partir de um conjunto de pontos (dispostos numa matriz), se não tratarmos de maneira adequada o vetor *“twist”* de cada segmento de superfície?

> Considerando $r(u,w)$, onde $u$ e $w$ são parâmetros da supercície $r$, $r_u$ é o vetor tangente à superfície na direção de $u$, e $r_w$ é o vetor tangente à superfície na direção de $w$. O vetor *“twist”* é a variação de $r_u$ em relação a $r_w$ e vice-versa.

> Para exemplificar o cuidado que o vetor *“twist”* demanda, caso um dos segmentos da superfície seja um cilindro, os vetores *“twist”* no fechamento do cilindro serão iguais, o que pode causar uma falsa impressão durante o equacionamento de que não há este segmento cilíndrico.

## b)

No desenvolvimento de um sistema gerador de trajetórias para o recobrimento de uma superfície (caminhos paralelos), para máquinas ferramentas e robôs manipuladores, quais especificações, em termos de modelagem de curvas e superfícies, devem ser consideradas e como é possível garantir o atendimento destas especificações? Discuta esta questão, através das técnicas de curvas e superfícies apresentadas avaliando como propriedades discutidas em aula podem contribuir ou não na programação desta tarefa.

> Considerando um caminho estabelecido por uma curva, os robôs manipuladores percorrem sequências de segmentos de retas, quanto menores os segmentos mais suave será o caminho, pois não será necessária mudança brusca de direção na transição de um segmento de reta para o outro. Por outro lado, armazenar uma grande quantidade de pontos não é interessante em termos de eficiência computacional, então uma solução interessante é estabelecer dois parâmetros para gerar a tragetória: o mínimo raio de curvatura e o máximo erro admissível. Com isso é possível diminuir consideravelmente a quantidade de pontos armazenados sem comprometer a suavidade da curva.

## c)

Considerando os temas e tópicos discutidos em toda a disciplina, discuta as aplicações em termos de vantagens e desvantagens do uso da teoria de curvas e superfícies em projeto e processos de fabricação.

> Vantagens: possibilidade de concepção de produtos com desenho não cartesiano; geração de trajetórias suaves para máquinas ferramentas; reprodução de um modelo físico para um modelo computacional através da coleta de coordenadas de pontos da superfície.

> Desvantagens: caso a curva possa ser obtida através de equação paramétrica é preferível em vez de obter através das técnicas como *Bezier* e *Bspline*; as trajetórias obtidas através da coleta de coordenadas de pontos da superfície poderão não ser totalmente fiéis à superfície existente, podendo levar a problemas dependendo da precisão requerida no processo.